<a href="https://colab.research.google.com/github/cozyrim/-Competition/blob/main/FoodCombine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random
import numpy as np
import csv

# 'FoodDataFrame.csv' 파일을 읽어옴. 데이터는 UTF-8로 인코딩되어 있다고 가정
nutrient_df = pd.read_csv('FoodDataFrame.csv', encoding='utf-8')

# NaN 값을 0으로 채움
nutrient_df = nutrient_df.fillna(0)

# 선택된 특징들을 정의
selected_features = ['식단', '선호도', '칼로리']

#확인용 출력
df = nutrient_df

In [ ]:
# 사용자로 부터 목표 칼로리 입력받기
selected_combinations = {}

total_calories = int(input("목표 칼로리를 입력하시오: "))

# 데이터프레임을 순회하며 목표 칼로리에 맞는 음식 조합을 선택
for index, row in nutrient_df.iterrows():
    calorie = row['칼로리']
    if total_calories - 50 <= calorie <= total_calories + 50:
        selected_meal = row['식단']
        selected_combinations[selected_meal] = {
            'preference': row['선호도'],
            'total_calories': calorie
        }

목표 칼로리를 입력하시오: 800


In [ ]:
# 선택된 음식 조합을 DataFrame에 추가
df_data = []
for dish, values in selected_combinations.items():
    df_data.append([dish, values['preference'], values['total_calories']])

df = pd.DataFrame(df_data, columns=['식단', '선호도', '칼로리'])

df # 중간 확인

,식단,선호도,칼로리
0,"크림스파게티, 전복탕, 깻잎김치",50,752.00
1,"쉬림프 로제 파스타, 내장탕, 꽈리고추멸치볶음",80,765.54
2,"채소볶음밥, 시금치된장국, 가자미조림",50,809.99
3,"<더 이지>동경규동, 소고기샤브샤브, 치킨텐더",50,765.75
4,"냉김치말이국수, 홍합미역국, 오리지널 포테이토 S",50,793.53
...,...,...,...
134,"메밀국수, 대구매운탕, 마늘쫑조림",50,777.86
135,"알밥, 쑥된장국, 순살화이트 어니언 치킨",50,817.77
136,"콩나물국밥, 닭개장, 쉑쉑치킨 (칠리)",50,799.24
137,"상하이쉬림프파스타, 어묵김치찌개, 닭꼬치",50,773.16


In [ ]:
# 사용자 선호도 학습 함수
def user_study(dataframe):
  count = 1
  while count == 1:
    state = random.choice(dataframe['식단'])
    print("Current Food:", state)
    while True:
      action = input("Enter 'O' for like, 'X' for dislike, or 'Q' to quit: ").strip().upper()

      if action == 'Q':
        print("\n")
        count = 0
        break
      elif action in ('O', 'X'):
        if action == 'O':
          dataframe.loc[dataframe['식단'] == state, '선호도'] += 1  # 선호도 증가
        else:
          dataframe.loc[dataframe['식단'] == state, '선호도'] -= 2  # 선호도 감소
        print("Updated Preference:", dataframe.loc[dataframe['식단'] == state, '선호도'].values[0])
        print()
        break
      else:
        print("Invalid menu. Select menu again.")

  return dataframe

In [ ]:
# 선호도 수정 함수
def update_preference(dataframe):
  index = int(input("수정할 행의 번호를 입력하세요: "))
  new_preference = int(input("새로운 선호도 값을 입력하세요: "))
  dataframe.loc[index, '선호도'] = new_preference
  return dataframe

In [ ]:
# 선호도가 특정 값 이상인 음식을 찾는 함수
def find_over_preference():
  preference = int(input("입력 값 보다 이상인 선호도를 찾습니다."))
  preferred_dishes = df[df['선호도'] >= preference]
  return preferred_dishes

In [ ]:
# 음식 추천 함수
def recommend_food(df, alpha=0.1, gamma=0.9, epsilon=0.2, num_episodes=1000):
    num_states = len(df)  # 음식 항목 수 (상태)
    num_actions = 2       # 좋아요 (O) 또는 싫어요 (X)

    q_table = np.zeros((num_states, num_actions))

    for episode in range(num_episodes):
        state = random.randint(0, num_states - 1)  # 무작위 상태에서 시작
        done = False

        while not done:
            action = np.argmax(q_table[state, :])  # 가장 높은 Q-값을 가진 행동 선택

            if random.uniform(0, 1) < epsilon:  # 탐색을 위한 약간의 무작위성 도입 (epsilon-탐욕 방식)
                action = random.randint(0, num_actions - 1)

            reward = df.loc[state, '선호도'] if action == 0 else -2 * df.loc[state, '선호도']

            next_state = state  # 설명을 위해 다음 상태가 현재와 동일하다고 가정

            # Q-학습 업데이트 규칙을 사용하여 Q-값 업데이트
            q_table[state, action] = (1 - alpha) * q_table[state, action] + \
                                     alpha * (reward + gamma * np.max(q_table[next_state, :]))

            state = next_state

            # 종료 조건 (설명을 위해)
            if random.uniform(0, 1) < 0.1:  # 에피소드 종료 확률을 10%로 설정
                done = True

    most_preferred_food_idx = np.argmax(np.max(q_table, axis=1))
    most_preferred_food = df.loc[most_preferred_food_idx, '식단']

    return most_preferred_food

In [ ]:
# 데이터프레임을 CSV 파일로 업데이트하는 함수를 정의
def update_csv_with_dataframe(filename, df):
    # 기존 CSV 파일을 DataFrame으로 읽어옴
    csv_df = pd.read_csv(filename, encoding = 'utf-8')

    # 기존 CSV 파일에서 DataFrame의 '식단'에 해당하는 열을 찾아 업데이트
    for index, row in df.iterrows():
        mask = csv_df['식단'] == row['식단']
        csv_df.loc[mask, '선호도'] = row['선호도']

    # 업데이트된 내용을 CSV 파일로 저장
    csv_df.to_csv(filename, encoding = 'utf-8', index=False)

In [ ]:
# 선호도가 40 이하인 음식을 삭제하는 함수
def delete_food():
  df = pd.read_csv('FoodDataFrame.csv', encoding='utf-8')
  df = df.fillna(0)
  df = df[df.선호도 >= 40]

  df.to_csv("FoodDataFrame.csv", index = False)

In [ ]:
# 메뉴를 선택할 수 있는 루프를 실행
while True:
  print("----------MENU----------")
  print("1.사용자 선호도 학습")
  print("2.선호도 수정")
  print("3.높은 선호도 검색")
  print("4.메뉴 추천")
  print("5.종료")
  print("------------------------")

  num = int(input("번호를 선택하시오."))
  print()

  if num == 1:
    df = user_study(df)
  elif num == 2:
    print(df)
    df = update_preference(df)
  elif num == 3:
    preferred_dish = find_over_preference()
    print(preferred_dish)
  elif num == 4:
    most_preferred_food = recommend_food(df)
    print("Most Preferred Food:", most_preferred_food)
  elif num == 5:
    update_csv_with_dataframe('FoodDataFrame.csv', df)
    delete_food()
    break
  else:
    print("올바른 메뉴를 선택해주세요")

----------MENU----------
1.사용자 선호도 학습
2.선호도 수정
3.높은 선호도 검색
4.낮은 선호도 삭제
5.메뉴 추천
6.종료
------------------------
번호를 선택하시오.5

Most Preferred Food: 쉬림프 로제 파스타, 내장탕, 꽈리고추멸치볶음
----------MENU----------
1.사용자 선호도 학습
2.선호도 수정
3.높은 선호도 검색
4.낮은 선호도 삭제
5.메뉴 추천
6.종료
------------------------
번호를 선택하시오.5

Most Preferred Food: 쉬림프 로제 파스타, 내장탕, 꽈리고추멸치볶음
----------MENU----------
1.사용자 선호도 학습
2.선호도 수정
3.높은 선호도 검색
4.낮은 선호도 삭제
5.메뉴 추천
6.종료
------------------------
번호를 선택하시오.5

Most Preferred Food: 샐러드김밥, 곰치국, 갓김치
----------MENU----------
1.사용자 선호도 학습
2.선호도 수정
3.높은 선호도 검색
4.낮은 선호도 삭제
5.메뉴 추천
6.종료
------------------------
번호를 선택하시오.5

Most Preferred Food: 불고기 크림 리소토, 서울식 불고기 전골, 고구마치즈볼
----------MENU----------
1.사용자 선호도 학습
2.선호도 수정
3.높은 선호도 검색
4.낮은 선호도 삭제
5.메뉴 추천
6.종료
------------------------
번호를 선택하시오.5

Most Preferred Food: 쉬림프 로제 파스타, 내장탕, 꽈리고추멸치볶음
----------MENU----------
1.사용자 선호도 학습
2.선호도 수정
3.높은 선호도 검색
4.낮은 선호도 삭제
5.메뉴 추천
6.종료
------------------------
번호를 선택하시오.5

Most Preferred Food: 쉬림프 로제 파스타, 내장탕, 